In [2]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install hopsworks

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-kybjk9gi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-kybjk9gi
  Resolved https://github.com/huggingface/transformers to commit df5c5c62ae253055336f5bb0828ca8e3e15ab6bd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8195433 sha256=4b02f172ba3fe5a68c54cda71104eb108ab0c1251c458552e8fb4dbf6c80deac
  Stored in directory: /tmp/pip-ephem-wheel-cache-r91aip9d/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_15_0", "zh-CN", split="train[:10%]", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_15_0", "zh-CN", split="test[:10%]", use_auth_token=True)

print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 5125it [00:00, 50874.88it/s]
Reading metadata...: 12653it [00:00, 65181.95it/s]
Reading metadata...: 29396it [00:00, 69355.77it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 7503it [00:00, 75018.98it/s]
Reading metadata...: 17483it [00:00, 89565.95it/s]
Reading metadata...: 30161it [00:00, 106546.10it/s]
Reading metadata...: 40816it [00:00, 104226.58it/s]
Reading metadata...: 51247it [00:00, 103551.05it/s]
Reading metadata...: 61928it [00:00, 104637.94it/s]
Reading metadata...: 72939it [00:00, 106396.72it/s]
Reading metadata...: 84306it [00:00, 108693.21it/s]
Reading metadata...: 95181it [00:00, 108349.71it/s]
Reading metadata...: 106020it [00:01, 107869.76it/s]
Reading metadata...: 117411it [00:01, 109702.13it/s]
Reading metadata...: 128385it [00:01, 107315.52it/s]
Reading metadata...: 140114it [00:01, 110270.15it/s]
Reading metadata...: 151156it [00:01, 105030.66it/s]
Reading metadata...: 161936it [00:01, 105825.54it/s]
Reading metadata...: 173093it [00:01, 107489.68it/s]
Reading metadata...: 183878it [00:01, 105755.43it/s]
Reading metadata...: 194802it [00:01, 106771.23it/s]
Reading metadata.

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 7104it [00:00, 71033.29it/s]
Reading metadata...: 17575it [00:00, 90835.90it/s]
Reading metadata...: 26659it [00:00, 76170.15it/s]
Reading metadata...: 38722it [00:00, 91885.97it/s]
Reading metadata...: 58157it [00:00, 94902.37it/s] 
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 2940
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 1062
    })
})


Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). Common Voice contains additional
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])



In [9]:
common_voice = common_voice.remove_columns("variant")
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2940
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1062
    })
})


In [10]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [11]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [12]:
input_str = common_voice["train"][0]["sentence"]
input_str = common_voice["test"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 黑身准裂腹鱼为辐鳍鱼纲鲤形目鲤科的其中一种。
Decoded w/ special:    <|startoftranscript|><|zh|><|transcribe|><|notimestamps|>黑身准裂腹鱼为辐鳍鱼纲鲤形目鲤科的其中一种。<|endoftext|>
Decoded w/out special: 黑身准裂腹鱼为辐鳍鱼纲鲤形目鲤科的其中一种。
Are equal:             True


### Combine To Create A WhisperProcessor

In [13]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")

### Prepare Data

In [14]:
print(common_voice["test"][0])
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/8374f15d5c3c5e127f4761aa5b6efdde55a55a27dab86543f709033739c8ba4b/zh-CN_test_0/common_voice_zh-CN_33411896.mp3', 'array': array([ 5.68434189e-14,  5.25801624e-13,  2.98427949e-13, ...,
        5.43916321e-05, -4.30148939e-05, -6.90988236e-05]), 'sampling_rate': 48000}, 'sentence': '黑身准裂腹鱼为辐鳍鱼纲鲤形目鲤科的其中一种。'}
{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/346a7cf7a5ce83376df8cee8c4a62128848956a0f1a002dbc38f03dcf330c18d/zh-CN_train_0/common_voice_zh-CN_22154274.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -1.62431343e-06, -6.40250619e-07,  1.10271367e-06]), 'sampling_rate': 48000}, 'sentence': '目前正在爆发的兹卡病毒疫情引发运动员和游客的担忧。'}


In [15]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [16]:
print(common_voice["test"][0])
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/8374f15d5c3c5e127f4761aa5b6efdde55a55a27dab86543f709033739c8ba4b/zh-CN_test_0/common_voice_zh-CN_33411896.mp3', 'array': array([ 1.45519152e-10,  6.18456397e-11, -7.27595761e-11, ...,
        4.87153593e-05,  6.62937382e-05, -6.62192178e-06]), 'sampling_rate': 16000}, 'sentence': '黑身准裂腹鱼为辐鳍鱼纲鲤形目鲤科的其中一种。'}
{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/346a7cf7a5ce83376df8cee8c4a62128848956a0f1a002dbc38f03dcf330c18d/zh-CN_train_0/common_voice_zh-CN_22154274.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        3.18077582e-06,  3.89223715e-07, -1.19029482e-06]), 'sampling_rate': 16000}, 'sentence': '目前正在爆发的兹卡病毒疫情引发运动员和游客的担忧。'}


In [17]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [18]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/2940 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1062 [00:00<?, ? examples/s]

In [19]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2940
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1062
    })
})

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
common_voice.save_to_disk(F"/content/drive/MyDrive/common_voice_15")

Saving the dataset (0/6 shards):   0%|          | 0/2940 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/1062 [00:00<?, ? examples/s]